In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
        super(SentimentRNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, 
                           batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x, hidden):
        # x shape: (batch_size, sequence_length)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out[:, -1, :]  # Get last time step output
        out = self.fc(lstm_out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        h0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size)
        c0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size)
        return (h0, c0)

# Example usage
vocab_size = 10000
embedding_dim = 100
hidden_size = 256
num_layers = 2

model = SentimentRNN(vocab_size, embedding_dim, hidden_size, num_layers)


In [8]:

# Sample input (batch of sequences of word indices)
batch_size = 16
sequence_length = 50
sample_input = torch.randint(0, vocab_size, (batch_size, sequence_length))
print('sample shape:', sample_input.shape)
hidden = model.init_hidden(batch_size)
print('hidden shape:', hidden[0].shape)
# Get predictions
output, hidden = model(sample_input, hidden)
print('output shape:', output.shape)
predictions = torch.sigmoid(output)  # Convert to probabilities

sample shape: torch.Size([16, 50])
hidden shape: torch.Size([2, 16, 256])
output shape: torch.Size([16, 1])


In [5]:
print(predictions.shape, output.size)
print(predictions)

torch.Size([16, 1]) <built-in method size of Tensor object at 0x105981260>
tensor([[0.4921],
        [0.5000],
        [0.4931],
        [0.4984],
        [0.4969],
        [0.4916],
        [0.4978],
        [0.4926],
        [0.4984],
        [0.4938],
        [0.4961],
        [0.4941],
        [0.4907],
        [0.4921],
        [0.4991],
        [0.4981]], grad_fn=<SigmoidBackward0>)
